# Aerospike Queries with Python¶


## Run the following cells to start the database and populate the test data.

### Start the server and verify the log.

In [ ]:
%%bash
asd
sleep 10
pgrep -x asd >/dev/null && echo "Aerospike server is running!" || echo "***Aerospike server is not running***"
echo "End of server log:"
tail -20 /tmp/aerolog

### Connect to the database and populate test data.

In [ ]:
# import the module
from __future__ import print_function
import aerospike

# Configure the client
config = {
  'hosts': [ ('127.0.0.1', 3000) ],
  'policy' : {'key': aerospike.POLICY_KEY_SEND}
}

# Create a client and connect it to the cluster
try:
  client = aerospike.client(config).connect()
except:
  import sys
  print("failed to connect to the cluster with", config['hosts'])
  sys.exit(1)

# Records are addressable via a tuple of (namespace, set, key)
people = [ {'id':1, 'name':'John Doe', 'age': 53},
           {'id':2, 'name':'Brian Yu', 'age': 21},
           {'id':3, 'name':'Will Kim', 'age': 34},
           {'id':4, 'name':'Dorothy Smith', 'age': 48},
           {'id':5, 'name':'Sara Poe', 'age': 29},
           {'id':6, 'name':'Kim Knott', 'age': 56},
           {'id':7, 'name':'Joe Miller', 'age': 30},
           {'id':8, 'name':'Jeff Nye', 'age': 32},
           {'id':9, 'name':'Jane Doe', 'age': 44},
           {'id':10, 'name':'Emily Tuck', 'age': 22} ]
try:
    for i in range(10):
      # Write the records
      client.put(('test', 'demo', 'id'+str(people[i])), people[i])
except Exception as e:
  import sys
  print("error: {0}".format(e), file=sys.stderr)

# Must create an index to query on a bin
try:
    client.index_integer_create("test", "demo", "age", "test_demo_number_idx")
except ex.IndexFoundError:
    pass

print('Database intialized.')

# Begin Query Tutorial

In addition to querying using the primary index using the key-value store APIs, the Aerospike Python client provides an API to query using secondary indexes.

## Query Records
Use the Aerospike Python client APIs to query the database using secondary indexes.

### Creating a Query
client.query() takes the namespace (required) and set (optional) arguments. set can be omitted or None. The return value is a new aerospike.Query class instance.

This example creates a query on the test namespace, demo set:

In [ ]:
query = client.query('test', 'demo')

### Projecting Bins
Project bins using select() on the Query class instance. select() accepts one or many bin names (strings).

This example selects name and age bins from the specified records:

In [ ]:
query.select('name', 'age')

### Adding Query Predicates
Define predicates using where() on the Query class instance. where() accepts a predicate created using one of the functions in aerospike.predicates, including:

equals(bin, value) — Find records containing bin bin with the specified value (integer or string).
between(bin, min, max) — Find records containing bin bin with a value in the min and max range (integer only).
Use p to import the predicates module from aerospike.predicates.

This example adds a between() predicate to a query:

In [ ]:
from aerospike import predicates as p
query.where( p.between('age', 14, 25) )

### Reading Results
Execute the query and read the results using foreach() in the Query class instance. foreach() accepts a callback function for each result read from the query. The callback function must accept a single argument as a tuple:

key tuple — The tuple to identify the record.
metadata — The dict containing the record metadata (TTL and generation).
record — The dict containing the record bins.
If the callback returns False, the client stops reading results.

These examples execute the query and reads results.

To print the records as they are read:

In [ ]:
def print_result(result_tuple):
    print(result_tuple)

To execute the query and call print_result for each result:

In [ ]:
query.foreach(print_result)

Clean up.

In [ ]:
# Close the connection to the Aerospike cluster
client.close()